<a href="https://colab.research.google.com/github/gogzicole/Hamoye-Data-science/blob/master/HAMOYE_STAGE_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [3]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.isnull().sum()

**counts the total number of values in target variable**

In [8]:
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [9]:
df = df.dropna()
df.head()
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

**converting to a binary classificatio problem by joining class 2A and 1A**

In [10]:
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [11]:
df_2A = df[df.QScore=='2A']
df_2A.count()


country           240
year              240
country_code      240
record            240
crop_land         240
grazing_land      240
forest_land       240
fishing_ground    240
built_up_land     240
carbon            240
total             240
QScore            240
dtype: int64

In [12]:
df_3A = df[df.QScore=='3A'].sample(350)
df_3A.count()

country           350
year              350
country_code      350
record            350
crop_land         350
grazing_land      350
forest_land       350
fishing_ground    350
built_up_land     350
carbon            350
total             350
QScore            350
dtype: int64

In [13]:
data_df = df_2A.append(df_3A)
data_df.count()

country           590
year              590
country_code      590
record            590
crop_land         590
grazing_land      590
forest_land       590
fishing_ground    590
built_up_land     590
carbon            590
total             590
QScore            590
dtype: int64

**shuffuling the dataset to randomize distribution of 2A and 3A**

In [14]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df['QScore'].value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

**dropping the first 3 columns and re-assigning QScore to y and the remaining data set to x**

In [ ]:
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
x = data_df.drop(columns='QScore')
y = data_df['QScore']


**splitting  data into test and split**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)
y_train.value_counts()

**now we must encode categorical data**

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

**if there is still an inbalance on the class distribution, for this we must use SMOTE to handle this, but only on the training data**

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)
x_train_balanced = DataFrame(x_train_balanced, columns=x_train.columns)

x_train_balanced

**now we perform feature scaling on the dataset**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']
#normalised_train_df

In [134]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']


**Logistics regression**

In [137]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)